## Back Test Results Binary Classification

In [ ]:
import pandas as pd
from datetime import datetime, timezone
from pycaret.datasets import get_data
from pycaret.classification import *
import yaml
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    pair = cfg['currency']['pair_2']
    dt = cfg['model']['dt']
    et = cfg['model']['et']
    knn = cfg['model']['knn'] 
    rf = cfg['model']['rf']
    ada = cfg['model']['ada']
    gbc = cfg['model']['gbc']
    svm = cfg['model']['svm']
    lightgbm = cfg['model']['lightgbm']
    lr = cfg['model']['lr']
    ridge = cfg['model']['ridge']
    lda = cfg['model']['lda']
    qda = cfg['model']['qda']
    nb = cfg['model']['nb']
    frac = cfg['model']['frac']
    random_state = cfg['model']['random_state']

In [ ]:
now = datetime.now()
today = now.strftime("%d-%m-%Y_%I-%M_%p")

In [ ]:
dataset = pd.read_csv('Back_Test_Result.csv')
dataset.shape

In [ ]:
dataset.head()

## Drop Predictions with Less than 20% LR Fit

In [ ]:
dataset = dataset[dataset['Current_Market_Fit'] > 20]
dataset = dataset.reset_index()
del dataset['index']

In [ ]:
dataset.shape

## One-Hot Encoding

In [ ]:
dataset['Current_Market'] = dataset['Current_Market'].replace(['Bullish','Bearish'],[1,0])
dataset['Rec1_Prediction'] = dataset['Rec1_Prediction'].replace(['BUY','SELL'],[1,-1])
dataset['Rec2_Prediction'] = dataset['Rec2_Prediction'].replace(['BUY','SELL'],[1,-1])
dataset['Rec3_Prediction'] = dataset['Rec3_Prediction'].replace(['BUY','SELL'],[1,-1])
dataset['Rec4_Prediction'] = dataset['Rec4_Prediction'].replace(['BUY','SELL'],[1,-1])

In [ ]:
dataset['Rec1_Score'] = dataset['Rec1_Score'] * dataset['Rec1_Prediction']
dataset['Rec2_Score'] = dataset['Rec2_Score'] * dataset['Rec2_Prediction']
dataset['Rec3_Score'] = dataset['Rec3_Score'] * dataset['Rec3_Prediction']
dataset['Rec4_Score'] = dataset['Rec4_Score'] * dataset['Rec4_Prediction']

In [ ]:
dataset = dataset.drop([
    'Candle_No',
    'Current_Market_Fit',
    'Rec1_Prediction',
    'Rec2_Prediction',
    'Rec3_Prediction',
    'Rec4_Prediction',
], axis=1)

In [ ]:
dataset.head()

In [ ]:
data = dataset.sample(frac=frac, random_state=random_state).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))
print('Fraction: ' + str(frac))
print('Random State: ', str(random_state))

# Create Unseen DataFrame

In [ ]:
data_unseen.head()

In [ ]:
exp_clf101 = setup(data = data, target = 'Current_Market', session_id=123, silent=True) 

In [ ]:
compare_models()

In [ ]:
best_model = gbc

In [ ]:
MODEL = create_model(best_model)

In [ ]:
TUNED_MODEL = tune_model(MODEL)

In [ ]:
plot_model(TUNED_MODEL, plot = 'auc')

In [ ]:
plot_model(TUNED_MODEL, plot = 'pr')

In [ ]:
plot_model(TUNED_MODEL, plot='feature')

In [ ]:
plot_model(TUNED_MODEL, plot = 'confusion_matrix')

In [ ]:
predict_model(TUNED_MODEL);

In [ ]:
FINAL_MODEL = finalize_model(TUNED_MODEL)

In [ ]:
print(FINAL_MODEL)

In [ ]:
predict_model(FINAL_MODEL);

In [ ]:
unseen_predictions = predict_model(FINAL_MODEL, data=data_unseen)
unseen_predictions.head()

In [ ]:
plot_model(FINAL_MODEL, plot = 'auc')

In [ ]:
pkl_name = (today + "_" + best_model + "_" + pair)

In [ ]:
save_model(FINAL_MODEL, pkl_name)

In [ ]:
SAVED_FINAL_MODEL = load_model(pkl_name)

In [ ]:
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)

In [ ]:
new_prediction.head(20)

In [ ]:
file = open(today + "_" + "binary_classification_log.txt", "w")
file.write ("Date: " + today + "\n" + \
            "Currency Pair: " + pair + "\n" + \
            "Model: " + best_model + "\n" + \
            "Frac: " + str(frac) + "\n" + \
            "Random State: " + str(random_state) + "\n" )
file.close()